Before running a <em>Chemkin</em> simulation, <i><u>mechanism data</u></i> must be first loaded and preprocessed. The mechanism data consist of a <u>mechanism input file</u>, a <u>thermodynamic data file</u>, and an optional <u>transport data file</u> which is required to run flame simulations. This example demonstartes the steps to properly preprocess the mechanism data before setting up any <em>Chemkin</em> simulations in Python. 

<h5>Step One</h5>Import the <strong>PyChemkin</strong> package and verify/set the working directory of the project

By default, the working directory is where this project file is located. Use methods in the <em>os</em> module such as <code>os.chdir(\<path>)</code> to switch to other directories. Turn ON/OFF the verbose mode in <em>Chemkin</em> by setting the value of <code>chemkin.verbose</code> to True/False.

In [ ]:
import os
import chemkin as ck                      # Chemkin

# check working directory
current_dir = os.getcwd()
print('current working directory: ' + current_dir)
# set verbose mode
ck.verbose = True

<hr>You can access the on-line Chemkin manuals by using the <ocde>help</code> method. Note that <strong>PyChemkin</code> will <I><u>try to open a new browser tab</u></i>, and you need to provide your credentials to enter the <u>Ansys Help portal</u>.   

In [ ]:
ck.help(topic='manuals')

<h5>Step Two</h5>Create a <em>Chemistry Set</em> by associating it with the mechanism data files

The first mechanism loaded is the GRI 3.0 mechanism for methane combustion. The mechanism and its associated data files come with the standard <em>Ansys Chemkin</em> installation under the subdirectory <b><i>"\reaction\data"</i></b>. A <em>Chemistry Set</em> <code>MyGasMech</code> is created as a new <code>chemkin.Chemistry</code> object with all the necessary input files assigned. The gas-phase mechanism input file, <i><u>chem</u></i>, is a required parameter. The thermodynamic data file, <i><u>therm</u></i>, is required only if the mechanism input file does not contain the thermodynamic data for all gas species. The transport data file, <i><u>tran</u></i>, on the other hand is optional; it is only required if the <em>Chemistry Set</em> is intended for flame simulations or for chemkin reactor models that involve transport processes. If there is no transport data file available, the <i><u>tran</u></i> parameter should be left out.   

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = ck.ansys_dir + '\\reaction\\data'
mechanism_dir = data_dir
# set mechanism input files
# inclusion of the full file path is recommended
chemfile = mechanism_dir+'\\grimech30_chem.inp'
thermfile = mechanism_dir+'\\grimech30_thermo.dat'
tranfile = mechanism_dir+'\\grimech30_transport.dat'
# create a chemistry set based on GRI 3.0
MyGasMech = ck.Chemistry(chem=chemfile, therm=thermfile, tran=tranfile, label='GRI 3.0')

<h5>Step Three</h5>Preprocess the <em>Chemistry Set</em>

Use the <code>chemkin.preprocess</code> method to <u>proprocess</u> the mechanism data. If a non-zero value is returned from the process, check the text output files <i><u>chem.out</u></i>, <i><u>tran.out</u></i>, or <i><u>summary.out</u></i> for potential error messages about the mechanism data. Otherwise, print out the mechanism sizes: the number of elements, <code>MM</code>; the number of gas species <code>KK</code>; the number of reactions, <code>IIGas</code>.

In [ ]:
# preprocess the mechanism files
iError = MyGasMech.preprocess()
print()
if iError != 0:
    print(f'PreProcess: error encountered...code = {iError:d}')
    print(f'see the summary file {MyGasMech.summaryfile} for details')
    exit()
else:
    print(ck.Color.GREEN + 'PreProcess success!!', end='\n' + ck.Color.END)
    print('mechanism information:')
    print(f'number of elements = {MyGasMech.MM:d}')
    print(f'number of gas species = {MyGasMech.KK:d}')
    print(f'number of gas reactions = {MyGasMech.IIGas:d}')


Use <em>Chemistry</em> methods <code>elementsymbols</code> and <code>AWT</code> to obtain <u>the symbol and the atomic mass of the elements</u> defined in the gas mechanism. List the element information in the order as they are defined in the mechanism.   

In [ ]:
print(f'\nelement and species information of mechanism {MyGasMech.label}')
# extract element symbols as a list
elelist = MyGasMech.elementsymbols
# get atomic masses as numpy 1D double array
AWT = MyGasMech.AWT
# print element information
for k in range(len(elelist)):
    print(f'element # {k+1:3d}: {elelist[k]:16} mass = {AWT[k]:f}')

Use <em>Chemistry</em> methods <code>speciessymbols</code> and <code>WT</code> to obtain <u>the symbol and the molecular mass of the gas species</u> defined in the gas mechanism. List the gas species information in the order as they are declared in the mechanism.

In [ ]:
# extract gas species symbols as a list
specieslist = MyGasMech.speciessymbols
# get species molecular masses as numpy 1D double array
WT = MyGasMech.WT
# print gas species information
for k in range(len(specieslist)):
    print(f'species # {k+1:3d}: {specieslist[k]:16} mass = {WT[k]:f}')

The second mechanism to be loaded into the project is the C2-NOx mechanism for the combustion of C1-C2 hydrocarbons. This mechanism differs from the GRI mechanism in the sense that it is self-contained, that is, the thermodynamic and the transport data of all species are included in the mechanism input file <i><u>C2_NOx_SRK.inp</u></i> which sits in the same reaction data directory as the GRI mechanism input files. Here the Chemistry Set <code>My2ndMech</code> is created without any input file parameter. The mechanism input file is instead assigned through the Chemistry class property <code>chemfile</code>. Because the C2-NOx mechanism does not have thermodynamic or the transport data file, there is no need to specify the thermodynamic and the transport data file names.   

<h5>Step One</h5> Create the <em>Chemistry Set</em> object and assign the mechanism input file 

In [ ]:
#
# load the second mechanism
#
# set the 2nd mechanism directory (the default chemkin mechanism data directory)
mechanism_dir = data_dir
# create a chemistry set based on C2_NOx using an alternative method
My2ndMech = ck.Chemistry(label='C2 NOx')
# set mechanism input files individually
# this mechanism file contains all the necessary thermodynamic and transport data
# therefore no need to specify the therm and the tran data files
My2ndMech.chemfile = mechanism_dir+'\\C2_NOx_SRK.inp'

By default, chemkin preprocessor would process the transport data only when the transport data file is specified. Since the C2-NOx mechanism does not need the additional transport data file, the transport data in the mechanism input file will not be processed automatically. In order to process the transport data in the mechanism file, the <code>preprocesstransportdata</code> method must be used prior to the preprocessing step. 

<h5>Step Two</h5> Instruct <em>Chemkin</em> preprocessor to also process the transport data in the mechanism file

In [ ]:
# instruct the preprocessor to include the transport properties
# only when the mechanism file contains all the transport data
My2ndMech.preprocesstransportdata()

<h5>Step Three</h5> <u>Preprocess</u> the <em>Chemistry Set</em> <code>My2ndMech</code>

Note that the C2-NOx mechanism also contains data to utilize the <u>Soave cubic equation of state (CuEOS) real-gas model</u> in <em>Chemkin</em>, and a message from <strong>PyChemkin</strong> will confirm the availability of the rea-gas model associated with this mechanism.

In [ ]:
# preprocess the 2nd mechanism files
iError = My2ndMech.preprocess()
print()
if iError != 0:
    print(f'PreProcess: error encountered...code = {iError:d}')
    print(f'see the summary file {My2ndMech.summaryfile} for details')
    exit()
else:
    print(ck.Color.GREEN + 'PreProcess success!!', end='\n' + ck.Color.END)
    print('mechanism information:')
    print(f'number of elements = {My2ndMech.MM:d}')
    print(f'number of gas species = {My2ndMech.KK:d}')
    print(f'number of gas reactions = {My2ndMech.IIGas:d}')

The third mechanism, <i><u>diesel_14comp_WBencrypt.inp</u></i>, is an encrypted mechanism. The <em>Chemistry Set</em> creation procecudres described above still spply, encrypted or not.  

<h5>Step One</h5> Specify the mechanism input file(s) with the full file path. This diesel mechanism is also located in the <i>reaction data</i> directory of the <em>Ansys Chemkin</em> installation. Normally, a mechanism should come with all the necessary data files or , like in this case, is self-contained.   

In [ ]:
#
# load the second mechanism
#
# set the 2nd mechanism directory (the default chemkin mechanism data directory)
mechanism_dir = data_dir
# create a chemistry set based on the encrypted 14 component diesel mechanism 
My3rdMech = ck.Chemistry(label='14comp diesel')
# set mechanism input files individually
# this mechanism file contains all the necessary thermodynamic and transport data
# therefore no need to specify the therm and the tran data files
My3rdMech.chemfile = mechanism_dir+'\\diesel_14comp_WBencrypt.inp'

<h5>Step Two</h5> <u>Preprocess</u> the mechanism. Note that the transport data is not processed because the <code>preprocesstransportdata</code> method is not called. 

In [ ]:
# preprocess the 3rd mechanism files
iError = My3rdMech.preprocess()
print()
if iError != 0:
    print(f'PreProcess: error encountered...code = {iError:d}')
    print(f'see the summary file {My3rdMech.summaryfile} for details')
    exit()
else:
    print(ck.Color.GREEN + 'PreProcess success!!', end='\n' + ck.Color.END)
    print('mechanism information:')
    print(f'number of elements = {My3rdMech.MM:d}')
    print(f'number of gas species = {My3rdMech.KK:d}')
    print(f'number of gas reactions = {My3rdMech.IIGas:d}')

<h5>Step Three</h5>Even the mechanism data is encrypted, basic properties of the species in the mechanism are available. For example, all of the 1883 species symbols and molecular weights are listed below.

In [ ]:
# extract gas species symbols as a list
specieslist = My3rdMech.speciessymbols
# get species molecular masses as numpy 1D double array
WT = My3rdMech.WT
# print gas species information
for k in range(len(specieslist)):
    print(f'species # {k+1:3d}: {specieslist[k]:16} mass = {WT[k]:f}')

The last mechanism is the full gasolie-diesel combustion mechanism from the <b>Model Fuel Library (MFL)</b>. As the word <i><u>full</u></i> implies, this mechanism consists of large number of species and reactions. The mechanism files are encrypted and are available under the <i><u>data\ModelFuelLibrary\Full</u></i> subdirectory. Since the MFL mechanisms are updated annually, the version number (the year) of the filenames varies from one release to another. Verify and update the MFL version number (if necessary) before running this tutorial.  

<h5>Step One</h5> Specify the mechanism files (full path) and create the <em>Chemistry Set</em> object

In [ ]:
mechanism_dir=data_dir+'\\ModelFuelLibrary\\Full'
# set the MFL mechanism version number 
MFLver = 'MFL2023'
print('MFL Full mechanism directory: '+mechanism_dir)
chemfile = mechanism_dir+'\\Diesel_PAH_NOx_chem_'+MFLver+'.inp'
thermfile = mechanism_dir+'\\Gasoline-Diesel-Biodiesel_PAH_NOx_therm_'+MFLver+'.dat'
tranfile = mechanism_dir+'\\Gasoline-Diesel-Biodiesel_PAH_NOx_tran_'+MFLver+'.dat'
# create a chemistry set based on the MFL full gasoline mechanism 
MyLargeMech = ck.Chemistry(chem=chemfile, therm=thermfile, tran=tranfile, label='MFL gasoline')

<h5>Step Two</h5> <u>Preprocess</u> the mechanism. Be patient, this is a large mechanism, the processing will take <em>a few (>3) minutes</em> to complete.

In [ ]:
# preprocess the MFL mechanism files
# the transport properties will be processed because the transport data file is given
iError = MyLargeMech.preprocess()
print()
if iError != 0:
    print(f'PreProcess: error encountered...code = {iError:d}')
    print(f'see the summary file {MyLargeMech.summaryfile} for details')
    exit()
else:
    print(ck.Color.GREEN + 'PreProcess success!!', end='\n' + ck.Color.END)
    print('mechanism information:')
    print(f'number of elements = {MyLargeMech.MM:d}')
    print(f'number of gas species = {MyLargeMech.KK:d}')
    print(f'number of gas reactions = {MyLargeMech.IIGas:d}')

This tutorial demonstrates various ways to create and preprocess a <em>Chemistry Set</em> in <strong>PyChemkin</strong>. Defining a <em>Chemistry Set</em> is the first (and required) step to build a simulation project. Once the <em>Chemistry Set</em> is successfully created, species properties such as enthalpy, specific heat capacity, and viscosity can be obtained by using appropriate methods under the <em>Chemistry</em> class. Furthermore, the <em>Chemistry Set</em> is the foundation of the <em>Mixture</em> object which is the <b><i>core component</i></b> of <strong>PyChemkin</strong>. 